$${\bf Восстановление\spaceполя\spaceна\spaceзаданной\spaceповерхности\spaceпо\spaceизмереням\spaceдальнего\spaceполя}$$

Пусть у нас имеются измерения поля на некоторой поверхности $\Sigma_M$, а также есть поверхность объема, содержащего внутри себя антенну, $\Sigma_R$. В этой области имеются неокторые источники токов $J$, $M$, по теорема эквивалентности, поле на поверхности $\Sigma_R$:$$E(J,M)=E(J_{eq}, M_{eq})$$ а вне ее $$E(J_{eq}, M_{eq})= 0$$
Удалим антенну и все источники, тогда задача относительно эквивалентных токов примет следующий вид:

Уравнение на поверхности, где произведены измерения $$ n\times E(r)= n\times[-\eta_0 L(J_{eq}, r) + K(M_{eq}, r)], r\in\Sigma_M $$

Уравнение на поверхности $\Sigma_R$ : $$ n\times[-\eta_0L(J_{eq},r)+K(M_{eq},r)]=-\frac{1}{2}M_{eq}(r), r\in \Sigma_R$$

где $L(\cdot)$ и $M(\cdot)$ соответствующие операторы интегрального представления электрического поля:
$$ L(J_{eq}, r)=\int_\Sigma \{\nabla \nabla'[J_{eq}G(r,r')]+k^2 J_{eq}G(r,r')\}dS' $$
$$ K(M_{eq}, r)=\int_\Sigma\{M_{eq}\times\nabla'G(r,r')\}dS'$$



Используя базисные функции RWG - $f_n$, искомые токи представим в виде $J_{eq} = \sum_{n=1}^N C_n^J f_n$ и $M_{eq} = \sum_{n=1}^N C_n^M f_n$. Применяя метод Галеркина, получим систему уравнений:
$$\begin{bmatrix}
-L_{MR}&\space\space -K_{MR} \\
-L_{RR}&\space\space K_{RR}+0.5I_{RR}
\end{bmatrix} 
\begin{bmatrix}
C^J\\
C^M
\end{bmatrix}=
\begin{bmatrix}
E_m\\
0
\end{bmatrix}
\hspace{3cm} [1]$$
где $E_m$ измерения дальнего поля, индексы блоков означают проекции операторов на соответствующие пространства: SR - RWG на пространство измерений, RR - RWG на пространство тестовых функций

$${\bf Постановка\spaceзадачи}$$

Имеется измерения дальнего поля антенны на сфере радиусом 1 км на частоте $0.7$ GHz. Требуется восстановить поле, излучаемое данной антенной на сфере радиусом 1 м.

Для дискретизации поставленной задачи используется BEMPP (https://bempp.com/handbook/api/boundary_operators.html). Здесь есть все необходимые операторы и базисные функции.
Обратим внимание, что количество измерений не совпадает с количеством ребер на сетке, заданной на поверхности, где требуется восстановление. Тогда для нахождения обратной матрицы системы воспользуемся методом наименьших квадратов, а для улучшения обусловленности матрицы - регуляризацией Тихонова.
Полученные коэффициенты - коэффициенты разложения искомых токов по базисным функциям: $J_{eq_n} = \sum_n C_n f_n$, следовательно восстановить поле можно по формуле: $ E_t = M_{eq}\times n$

(https://bempp-cl.readthedocs.io/en/latest/docs/bempp/api/)

Для начала работы установите следующие пакеты:

In [118]:
import numpy as np
import matplotlib.pyplot as plt
import bempp.api
import matplotlib as mpl
import plotly.figure_factory as ff
import plotly.graph_objects as go
from bempp.api.operators.potential import maxwell

Понадобятся следющие константы:

In [119]:
frequency = 0.7 * 10**9 
vacuum_permittivity = 8.854187817E-12
vacuum_permeability = 4 * np.pi * 1E-7
k_ext = 2 * np.pi * frequency * np.sqrt(vacuum_permittivity * vacuum_permeability)
wave_length = 2 * np.pi / k_ext

Для составления блоков матрицы системы $[1]$ понадобится вспомогательная функция, вычсиляющая блок матрицы, соответствующий потенциальному оператору, по сеточной функции:

In [120]:
#  pot_op - потенциальный оператор bempp
#  field_mat - матрица, вычсиляемая путем применения оператора к сеточной функции
def get_mat(pot_op):
    space = pot_op.space
    
    field_mat = []
    for i in range(space.global_dof_count):
        coef = np.zeros(space.global_dof_count)
        coef[i] = 1
        field = pot_op * bempp.api.GridFunction(space, coefficients=coef)
        field_vec = field.flatten(order='F')
        field_mat.append(field_vec[:, None])
        
    field_mat = np.hstack(field_mat)
    
    return field_mat

$${\bf План\space выполнения}$$
Предлагается написать метод, выполняющий восстановление поля на поверхности <span style="color:green">def</span> <span style="color:blue">current_reconstruction</span>(*) $[2]$, принимающий на вход:
1. файл с сеткой, файл из измерениями дальнего поля
2. волновое число
3. параметр для регуляризации,

а на выходе:
1. координаты центров элементов сетки, на котрой произведено восстановление
2. компоненты нормали
3. реальная часть компонент поля Е
4. мнимая часть компонент поля Е


Для удобства будем использовать шаблон класса, в котором необходимо дописать некотрые методы:

In [121]:
# Шаблон класса и его методы
class Current_reconstruction_class:
    def __init__(self, wave_number):
        self.wave_number = wave_number
        self.mtrx = None
        self.b = None

    # Определение пространств базисных и тестовых функций на сетке, где требуется восстановление:
    def set_reconstruction_grid(self, grid):
        self.reconstr_grid = grid
        self.reconstr_RWG = bempp.api.function_space(self.reconstr_grid, "RWG", 0)
        self.reconstr_SNC = bempp.api.function_space(self.reconstr_grid, "SNC", 0)

    # Определение начальных данных: значения известного дальнего поля и точек, в которых оно было измерено:
    def set_measured_field(self, points, electric_field):
        self.measur_efield = electric_field
        self.measur_points = points

    # Cборка правой части системы:
    def assemble_rhs(self):
        RHS = np.concatenate([
            self.measur_efield.flatten(order='F'), 
            np.zeros(self.reconstr_SNC.global_dof_count)
        ])
        self.rhs = RHS

    # Сборка левой части системы $[1]$. Допишите функцию с помощью операторов Bempp:
    def assemble_matrix(self, correction=True):
        # TODO
        L_SR = None
        K_SR = None
        # L_SR_mat = get_mat(L_SR)
        # K_SR_mat = get_mat(K_SR)

        # L_RR = maxwell.electric_field(
        #     space=self.reconstr_RWG, points=self.reconstr_grid.centroids, wavenumber=self.wave_number
        # )
        L_RR = None
        K_RR = None

    # Нахождение решения системы методом наименьших квадратов с регуляризацией Тихонова. Допишите метод.
    def reconstruct_currents(self, method="Least Squares", Tikhonov_reg=0.0):
        n = self.mtrx.shape[1]

        reg_matrix = np.vstack((self.mtrx, Tikhonov_reg * np.eye(n)))
        reg_b = np.vstack((self.b, np.zeros(shape=(n, self.b.shape[1]))))

        x = np.linalg.lstsq(reg_matrix, reg_b)
        return x[0]

    # Вычисление тангенциальных составляющих полей. Функция вовзращает центры элементов, нормали,E_t, H_t.
    # Допишите метод, используя методы класса GridFunction из Bempp:
    def evaluate_tangential_trace_on_element_centers(self):
        coeff = np.sqrt(vacuum_permeability / vacuum_permittivity)

        # TODO
        M_trace = None
        E_trace = None
        centers = None
        normals = None

        return centers, normals, E_trace, M_trace

    # Метод, использующий внутри себя все описанные выше функции (детали см. в План выполнения).
    # В качестве параметра регуляризации возьмите 0.001. Допишите метод:
    def current_reconstruction(grid_filename, data_filename, result_filename, k_ext, reg_parameter=1e-3):
        # TODO
        pass

### Create grid

In [122]:
mesh_path = "Sphere_1m/Sphere_1m_1269edges.msh"
grid = bempp.api.import_grid(mesh_path)

### Reconstruction

In [123]:
reconstruction = Current_reconstruction_class(k_ext)
reconstruction.set_reconstruction_grid(grid)

### Test Least Squares with Tikhonov regularization

In [128]:
#   mtrx size
n = 5
#   tests count
count = 10

for i in range(1, count + 1, 1):
    A = np.random.rand(n, n)
    b = np.random.rand(n, n)

    reconstruction.mtrx = A
    reconstruction.b = b

    x = reconstruction.reconstruct_currents(Tikhonov_reg=0.01)
    error = np.linalg.norm(A.dot(x) - b)

    print(f'test{i}: x={x}, error={error}')

test1: x=[[ -2.08616257  -8.20388102  -2.3186534   -5.38100191  -9.54748589]
 [ -3.07903704 -13.2703974   -2.99146162  -9.3782042  -12.91651499]
 [ -0.90882834  -5.65254357  -0.73274416  -5.10429409  -7.13967868]
 [  1.63445779   5.04509719   1.94294858   3.37769777   5.42528771]
 [  3.97893224  15.01622575   2.89269697  11.32610659  16.34800031]], error=0.32383405888619554
test2: x=[[-0.36530301 -0.24564427 -0.01921384 -0.47676107 -0.11296208]
 [ 1.13131176  0.5506407   1.28678393  0.24700451 -1.05264629]
 [ 0.70180067  1.0651869   0.17606124  0.03449748  0.24484297]
 [-0.16231408 -0.31522547 -0.45831887 -0.09140871  1.36091394]
 [ 0.65839953  0.56411022  0.17020663  0.79798429  0.96653076]], error=0.000849001399842241
test3: x=[[ 0.65845401  1.49309173  1.54507082 -2.08374387  1.16812513]
 [-0.23072851  0.80866421  1.35398663 -5.72272679  0.79723917]
 [-0.13780868  0.87374943  0.55606846 -3.91800021  0.38475597]
 [-0.25000369 -1.15277388 -0.77681185  6.03884829 -1.06964119]
 [ 0.6913

/var/folders/5r/6kgbywtn30b4vz8fkmqqhx2h0000gn/T/ipykernel_93555/837615966.py:48: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x = np.linalg.lstsq(reg_matrix, reg_b)


In [125]:
# from bempp.api.operators.boundary import maxwell

# elec = bempp.api.operators.boundary.maxwell.electric_field(
#     reconstruction.reconstr_RWG,
#     reconstruction.reconstr_RWG,
#     reconstruction.reconstr_SNC,
#     k_ext,
# )
# identity = bempp.api.operators.boundary.sparse.identity(
#     reconstruction.reconstr_RWG,
#     reconstruction.reconstr_RWG,
#     reconstruction.reconstr_SNC,
# )

# elec.strong_form()

# # elec.weak_form()

$$\bf Общая\spaceструктура \space работы$$

Мы детально разобрали, какие методы понадобятся для решения поставленной задачи. 
Сетка, на которой требуется восстановить токи - <span style="color:green">"test_data/test_data/Sphere_R1_1269edges.msh"</span>, измерения вы найдете в файле - <span style="color:green">"test_data/Ef_Port1_0_7GHz_Sphere_1km.csv"</span>, и референсные значения - <span style="color:green">"test_data/Ef_Port1_0_7GHz_Sphere_1m.csv"</span>. 
Итак, подытожим план выполнения практического задания:
1. С помощью метода Галеркина составить систему уравнений для восстановления эквивалентных токов $J_{eq_n}, M_{eq}$ по известным измерениям $E_m$
2. Используя операторы и пространства в Bempp (https://bempp.com/handbook/api/boundary_operators.html), собрать левую и правую части системы: <span style="color:blue">assemble_matrix</span> , <span style="color:blue">assemble_rhs</span>
3. Найти обратную матрицу системы с помощью МНК и регуляризации Тихонова: <span style="color:blue">reconstruct_currents</span>.
4. По найденным коэффициентам разложения токов найти тангенциальную составляющую поля $E$ на заданной поверхности: <span style="color:blue">current_reconstruction</span>
6. Сравнить полученные результаты с референсом